## 通过 Qwen Agent 操作 MySQL 数据库

[servers](https://github.com/modelcontextprotocol/servers) 没有提供官方 MySQL MCP Server 实现，我们用社区实现的版本 [designcomputer/mysql_mcp_server](https://github.com/designcomputer/mysql_mcp_server)。

### 一、安装 MySQL 数据库

在 Ubuntu 系统安装 MySQL 的过程如下：

**1）更新系统包列表**

```bash
sudo apt update
```

**2）安装 MySQL**

```bash
sudo apt install mysql-server
```

**3）运行安全配置向导**

运行以下脚本以启动安全配置向导：

```bash
sudo mysql_secure_installation
```

> `mysql_secure_installation` 是一个用于增强 MySQL 安全性的脚本。它会引导你完成一系列安全设置，解决安装新数据库的安全风险。

**4）启动 MySQL 服务**

```bash
# 启动 MySQL 服务
sudo systemctl start mysql

# 允许 MySQL 在启动时自动启动
sudo systemctl enable mysql
```

**5）检查 MySQL 状态**

```bash
sudo systemctl status mysql
```

**6）连接到 MySQL**

以 root 用户身份登录 MySQL：

```bash
sudo mysql -u root -p
```

创建新用户：

```mysql
CREATE USER 'username'@'localhost' IDENTIFIED BY 'password';
GRANT ALL PRIVILEGES ON *.* TO 'username'@'localhost' WITH GRANT OPTION;
FLUSH PRIVILEGES;
EXIT;
```

配置文件位于 `/etc/mysql/mysql.conf.d/mysqld.cnf`，可以修改 port, bind-address, buffer sizes 等设置。修改配置后，重启 MySQL：

```bash
sudo systemctl restart mysql
```

### 二、添加样例数据 

```mysql
-- 创建新数据库
CREATE DATABASE score;

-- 创建新用户
CREATE USER 'admin'@'localhost' IDENTIFIED BY 'admin-password';

-- 授予用户
GRANT ALL PRIVILEGES ON score.* TO 'admin'@'localhost' WITH GRANT OPTION;
FLUSH PRIVILEGES;

-- 切换数据库
USE score;

-- 创建学生表（包含表注释和字段注释）
CREATE TABLE students (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50) COMMENT '学生姓名',
    class VARCHAR(20) COMMENT '班级名称',
    gpa DECIMAL(3,1) COMMENT '学生绩点'
) COMMENT '存储学生信息的表';

-- 检查表是否成功创建
DESC score.students;

-- 插入 20 条样例数据
INSERT INTO students (name, class, gpa) VALUES
('张三', 'Class 1', 3.8),
('李四', 'Class 1', 3.5),
('王五', 'Class 1', 3.2),
('赵六', 'Class 1', 4.0),
('张万福', 'Class 2', 3.6),  -- 姓氏重叠示例
('钱七', 'Class 2', 2.8),
('李四', 'Class 2', 3.4),
('孙八', 'Class 2', 3.9),
('周九', 'Class 2', 3.1),
('吴十', 'Class 3', 2.5),
('王九幽', 'Class 3', 3.7),  -- 姓氏重叠示例
('郑十一', 'Class 3', 4.0),
('冯十二', 'Class 3', 3.3),
('陈十三', 'Class 3', 2.9),
('褚十四', 'Class 1', 3.5),
('卫十五', 'Class 2', 3.0),
('蒋十六', 'Class 3', 3.8),
('沈十七', 'Class 1', 2.7),
('韩十八', 'Class 2', 3.6),
('杨十九', 'Class 3', 3.2);

-- 检查数据是否成功插入
SELECT * FROM score.students;
```

### 三、Python 连接 MySQL




In [1]:
# !uv pip install mysql-connector-python

In [3]:
import mysql.connector
from mysql.connector import Error

try:
    # Establish connection
    connection = mysql.connector.connect(
        host='localhost',
        database='score',
        user='admin',
        password='admin-password',
        port=3306
    )

    if connection.is_connected():
        print("Successfully connected to MySQL")

        # Create cursor object
        cursor = connection.cursor()

        # Execute query
        query = (
            "SELECT *"
            "FROM score.students"
        )
        cursor.execute(query)
        record = cursor.fetchall()
        print(f"{record}")

except Error as e:
    print(f"Error: {e}")

finally:
    # Close connections
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed")


Successfully connected to MySQL
[(1, '张三', 'Class 1', Decimal('3.8')), (2, '李四', 'Class 1', Decimal('3.5')), (3, '王五', 'Class 1', Decimal('3.2')), (4, '赵六', 'Class 1', Decimal('4.0')), (5, '张万福', 'Class 2', Decimal('3.6')), (6, '钱七', 'Class 2', Decimal('2.8')), (7, '李四', 'Class 2', Decimal('3.4')), (8, '孙八', 'Class 2', Decimal('3.9')), (9, '周九', 'Class 2', Decimal('3.1')), (10, '吴十', 'Class 3', Decimal('2.5')), (11, '王九幽', 'Class 3', Decimal('3.7')), (12, '郑十一', 'Class 3', Decimal('4.0')), (13, '冯十二', 'Class 3', Decimal('3.3')), (14, '陈十三', 'Class 3', Decimal('2.9')), (15, '褚十四', 'Class 1', Decimal('3.5')), (16, '卫十五', 'Class 2', Decimal('3.0')), (17, '蒋十六', 'Class 3', Decimal('3.8')), (18, '沈十七', 'Class 1', Decimal('2.7')), (19, '韩十八', 'Class 2', Decimal('3.6')), (20, '杨十九', 'Class 3', Decimal('3.2'))]
MySQL connection closed


### 四、MCP 调用 MySQL

运行代码前，需要确保 LLM 和 PostgreSQL 正常运行。

要启动 LLM 服务，来到项目根目录运行：

```bash
cd test_qwen3
bash vllm_server.sh
```

In [4]:
from qwen_agent.agents import Assistant
from qwen_agent.gui import WebUI

def init_agent_service():
    llm_cfg = {
        'model': 'Qwen3-0.6B-FP8',
        'model_server': 'http://localhost:8000/v1',
        'api_key': 'token-kcgyrk',
        'generate_cfg': {
            'top_p': 0.95,
            'temperature': 0.6,
        }
    }
    system = ('你扮演一个数据库助手，你具有查询数据库的能力')
    tools = [{
      "mcpServers": {
        "mysql": {
          "type": "stdio",
          "command": "uvx",
          "args": [
              "--from",
              "mysql-mcp-server",
              "mysql_mcp_server"
          ],
          "env": {
            "MYSQL_HOST": "localhost",
            "MYSQL_PORT": "3306",
            "MYSQL_USER": "admin",
            "MYSQL_PASSWORD": "admin-password",
            "MYSQL_DATABASE": "score"
          }
        }
      }
    }]
    bot = Assistant(
        llm=llm_cfg,
        name='数据库助手',
        description='数据库查询',
        system_message=system,
        function_list=tools,
    )

    return bot

In [5]:
# 初始化 Agent
bot = init_agent_service()

2025-05-26 15:05:57,815 - mcp_manager.py - 110 - INFO - Initializing MCP tools from mcp servers: ['mysql']
2025-05-26 15:05:57,837 - mcp_manager.py - 245 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: mysql


In [6]:
query = '查询所有王姓同学的GPA平均分'
messages = [{'role': 'user', 'content': query}]

# 输出
response = bot.run_nonstream(messages)
print('bot response:', response)

bot response: [{'role': 'assistant', 'content': '', 'reasoning_content': "\n好的，用户需要查询所有王姓同学的GPA平均分。首先，我需要确定如何用MySQL来执行这个查询。用户提供的工具中有一个mysql-execute_sql函数，可以执行SQL查询。所以，我应该用这个函数来执行相应的SQL语句。\n\n接下来，我需要构造正确的SQL查询。用户要求的是所有王姓同学的GPA平均分。这里需要注意，可能需要使用GROUP BY来处理姓氏，但用户的问题中没有提到分组，所以可能直接查询所有王姓学生。不过，通常在数据库中，如果只是查找特定姓氏的记录，可能不需要分组，但为了准确，可能需要加上GROUP BY。但用户的问题比较笼统，直接查询所有王姓学生是否更合适。或者，可能需要考虑是否应该按姓氏分组，但如果没有分组，可能直接查询所有王姓学生的记录，然后计算平均分。因此，正确的SQL应该是SELECT AVG(gpa) FROM students WHERE name LIKE '王%'; 这样就能得到所有王姓学生的平均GPA。\n\n然后，我需要确认是否需要使用GROUP BY，但这里可能不需要，因为只需要所有记录的平均值。因此，调用mysql-execute_sql函数，传入这个查询语句。确保参数正确，没有语法错误。最后，返回工具调用的JSON结构。\n", 'name': '数据库助手'}, {'role': 'assistant', 'content': '', 'name': '数据库助手', 'function_call': {'name': 'mysql-execute_sql', 'arguments': '{"query": "SELECT AVG(gpa) AS average_gpa FROM students WHERE name LIKE \'王%\'"}'}}, {'role': 'function', 'content': 'average_gpa\n3.45000', 'name': 'mysql-execute_sql'}, {'role': 'assistant', 'content': '', 'reasoning_content': '\n好的，用户之前让我查询所有王姓

In [7]:
print('result:', response[-1]['content'].strip())

result: 王姓同学的GPA平均分为3.45。请确认是否需要进一步查询其他信息或有其他问题。
